# Faster interpolation and transformation using the GPU

Gryds has GPU support for the interpolation and deformable transformations. From our own testing, we know that this can give a significant speed improvement for 3D images.

By executing the interpolation on the GPU, we obtain a 2-3 times speed-up. Shown below is a curve for the number of seconds of GPU vs CPU linear interpolation as a function of image size.

The GPU execution has some overhead, that means that for smaller images (i.e. 2D images) the CPU implementation is still faster. Starting from an image size of 6 x 128 x 128 the GPU version is significantly faster.

![](cupy_interpolation.png)

If we also run the *transformation* on the GPU, the speed-up is even larger: about seven times faster.

![](cupy_transformation.png)


## Using GPU acceleration

The GPU versions of the interpolation and transformations are called `BSplineInterpolatorCuda` and `BSplineTransformationCuda`. Currently, only zero and first-order B-splines are supported on the GPU.

Except for a CUDA installation and an Nvidia GPU, there is one extra requirement: a package called `cupy`. `cupy` can be installed using `pip` by typing `pip install cupy`. Once installed, Gryds will automatically import the `BSplineInterpolatorCuda` and `BSplineTransformationCuda` classes, that you can use exactly like the normal `BSplineInterpolator` and `BSplineTransformationCuda` classes. I.e., to turn this CPU-only code

```python
image = np.random.rand(128, 128, 128)
tf = gryds.BSplineTransformation(0.01 * (np.random.rand(3, 32, 32, 32) - 0.5), order=1)
intp = gryds.BSplineInterpolator(image, order=1)
output = intp.transform(tf)
```

into GPU code, you just append `Cuda` to the class names:

```python
image = np.random.rand(128, 128, 128)
tf = gryds.BSplineTransformationCuda(0.01 * (np.random.rand(3, 32, 32, 32) - 0.5), order=1)
intp = gryds.BSplineInterpolatorCuda(image, order=1)
output = intp.transform(tf)
```

## Implementation details

These GPU versions of the interpolator and transformer use CUDA implementations of the `scipy.ndimage.map_coordinates()` function, which is called `cupyx.scipy.ndimage.map_coordinates()`. At the time of writing, this function only support zero and first-order B-spline interpolation, but future versions could also include higher-orders. Documentation on `cupy` can be found [here](https://docs-cupy.chainer.org/en/stable/reference/generated/cupyx.scipy.ndimage.map_coordinates.html#cupyx.scipy.ndimage.map_coordinates).